In [6]:
import torch
import torch.nn as nn
B = 1
in_channels = 55
in_channels_anatomy = 55
q_feat = torch.randn(B, in_channels_anatomy, 24,24,24) # B, C, H, W, D
q = q_feat.flatten(2).transpose(1,2) # B, H*W*D, C
print(q.shape)
print(q[0,0,:])
norm1 = nn.LayerNorm(in_channels)
q_normed = norm1(q)
print(q_normed.shape)
print(q_normed[0,0,:])

torch.Size([1, 13824, 55])
tensor([ 0.2678, -0.1319,  0.9623, -1.5242,  2.3663, -0.0973,  0.6046,  0.1900,
        -0.9018,  2.2388, -0.1959, -0.7357, -0.1842, -0.8305,  0.3882,  0.4091,
        -1.0197, -0.4825,  1.2117,  0.0866, -0.1653, -0.7874,  1.7620, -1.2261,
         0.5610, -1.2548, -0.1477,  0.4335,  0.9782,  0.1961,  0.1949,  0.3233,
         0.3969, -1.1417, -0.9993, -0.0609, -1.3078, -0.7852,  0.5121,  0.7289,
        -0.7256, -1.8970, -0.5562, -1.3670, -1.7766,  0.9565, -2.6712, -0.8030,
        -1.5722,  0.6578,  0.5076,  1.5423, -0.4131, -0.2812, -1.4708])
torch.Size([1, 13824, 55])
tensor([ 4.1994e-01,  3.1514e-02,  1.0948e+00, -1.3215e+00,  2.4592e+00,
         6.5150e-02,  7.4716e-01,  3.4428e-01, -7.1664e-01,  2.3352e+00,
        -3.0669e-02, -5.5527e-01, -1.9278e-02, -6.4741e-01,  5.3695e-01,
         5.5723e-01, -8.3123e-01, -3.0922e-01,  1.3371e+00,  2.4379e-01,
        -9.1915e-04, -6.0548e-01,  1.8719e+00, -1.0318e+00,  7.0483e-01,
        -1.0597e+00,  1.6167e

In [3]:
import torch
B =1
pathology_feat = torch.randn(B, 2, 24,24,24) # B, 2, H, W, D
anatomy_feat = torch.randn(B, 1, 24,24,24) # B, 2, H, W, D
pathology_ct = pathology_feat[:,:1,...]
print(f"pathology ct shape:{pathology_ct.shape}")
## assume cross_attention
enhanced_pathology = torch.cat([pathology_ct, pathology_feat[:,1:,...]], dim=1)
print(f"enhanced_pathology shape:{enhanced_pathology.shape}")


pathology ct shape:torch.Size([1, 1, 24, 24, 24])
enhanced_pathology shape:torch.Size([1, 2, 24, 24, 24])


In [12]:
import torch.nn as nn
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from omegaconf import DictConfig
from loguru import logger

class MirrorUNet(nn.Module):
    def __init__(self):
        super().__init__()
        norm = "instance"
        self.pathology_model = UNet(
            spatial_dims=3,
            in_channels=2,
            out_channels=2,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            num_res_units=2,
            norm=norm,
        )
        self.anatomy_model = UNet(
            spatial_dims=3,
            in_channels=1, # ct
            out_channels=145,
            channels=(32,64,128,256,512,1024),
            strides=(2,2,2,2,2),
            # num_res_units=num_res_units,
            norm="instance",
        )
            
m = MirrorUNet()
m.pathology_model.model 

Sequential(
  (0): ResidualUnit(
    (conv): Sequential(
      (unit0): Convolution(
        (conv): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
        (adn): ADN(
          (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
      (unit1): Convolution(
        (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (adn): ADN(
          (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
    )
    (residual): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  )
  (1): SkipConnection(
    (submodule): Sequential(
      (0): ResidualUnit(
        (conv): Sequential(
          (unit0): Convolution(
            (conv): Conv3

In [17]:
count = 0
for i, layer in enumerate(m.anatomy_model.model):
    print(f"layer {i}: {layer}")

layer 0: Convolution(
  (conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (adn): ADN(
    (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (D): Dropout(p=0.0, inplace=False)
    (A): PReLU(num_parameters=1)
  )
)
layer 1: SkipConnection(
  (submodule): Sequential(
    (0): Convolution(
      (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
      (adn): ADN(
        (N): InstanceNorm3d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (D): Dropout(p=0.0, inplace=False)
        (A): PReLU(num_parameters=1)
      )
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): Convolution(
          (conv): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): InstanceNorm3d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
            (D): Dropout(p=0.0, in

In [29]:
from monai.networks.layers import Norm, SkipConnection
count = 0
for i, layer in enumerate(m.pathology_model.model):
    # print(f"layer {i}: {layer}")
    print(f"-------")
    if i == 1:
        if isinstance(layer, SkipConnection):
            
            print(f"last bottelneck: {layer.submodule[1].submodule[1].submodule[1].submodule[1].submodule}") # skipconnection + skipconnection + ... + skipconnection + bottleneck
            print(f"first decoder: {layer.submodule[2][0]}")
            
      

-------
-------
first decoder: Convolution(
  (conv): ConvTranspose3d(128, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1))
  (adn): ADN(
    (N): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (D): Dropout(p=0.0, inplace=False)
    (A): PReLU(num_parameters=1)
  )
)
last bottelneck: ResidualUnit(
  (conv): Sequential(
    (unit0): Convolution(
      (conv): Conv3d(512, 1024, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (adn): ADN(
        (N): InstanceNorm3d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (D): Dropout(p=0.0, inplace=False)
        (A): PReLU(num_parameters=1)
      )
    )
    (unit1): Convolution(
      (conv): Conv3d(1024, 1024, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (adn): ADN(
        (N): InstanceNorm3d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (D): Dropout(p=0.0, i